In [1]:
# echo server program
import socket
import time
import threading
import os

# symbolic name meaning all available interfaces
HOST = 'localhost' 
# arbitrary non-privileged port
PORT = 10004  
# create the socket connection
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# bind the host and port to the socket
s.bind((HOST, PORT))
# listen to the client
s.listen(3)

In [5]:
# get the messages from the clients
def getMessages():
    # get the incoming connection request and the address of the client
    conn, addr = s.accept()
    conn.send(b'Server Connected...')
    full_data = b''
    
    while True:
        # get the data from the client in chunks
        data = conn.recv(4096)
        
        # get full data
        full_data = full_data + data
            
        #check if data is empty or not, if empty exit the while loop
        if b'?end' in data:
            ack = 'Hostname: ' + addr[0] + ', Port: ' + str(addr[1]) + ': File uploaded Sucessfully.'
            conn.send(ack.encode())
            break
            
    file_name = conn.recv(128).decode()
    # print the recived message from the client
    print('File Recived from client' + ' Hostname: ' + addr[0] + ', Port: ' + str(addr[1]) + ' at {0}.'.format(time.ctime(time.time())), end="\n")
               
    
    # close the client connection once the all the message is recieved
    conn.close()
    
    full_data = full_data[:-4]
    
    with open('UploadedFiles/' + file_name, 'wb') as f:
            f.write(full_data)
            f.close()
            
    # sleeping for 5 seconds
    time.sleep(5)

### Synchronous method

In [6]:
getMessages()
getMessages()
getMessages()

File Recived from client Hostname: 127.0.0.1, Port: 33142 at Tue Jun  8 18:43:34 2021.
File Recived from client Hostname: 127.0.0.1, Port: 33144 at Tue Jun  8 18:43:44 2021.
File Recived from client Hostname: 127.0.0.1, Port: 33146 at Tue Jun  8 18:43:49 2021.


### Asynchronous Method

In [4]:
# using threads to run the function getMessages
# creating the threads list to hold the threads
threads = []

# creating 3 threads for the clients
for _ in range(3):
    t = threading.Thread(target=getMessages)
    t.start()
    threads.append(t)

# closing threads once all the threads run
for thread in threads:
    thread.join()

File Recived from client Hostname: 127.0.0.1, Port: 48258 at Tue Jun  1 22:56:05 2021.
File Recived from client Hostname: 127.0.0.1, Port: 48254 at Tue Jun  1 22:56:05 2021.
File Recived from client Hostname: 127.0.0.1, Port: 48256 at Tue Jun  1 22:56:24 2021.


In [23]:
# close the socket connections
s.close()